# Cytoscape Node `FocusView`
TODO

In [ ]:
import rdflib

from ipyradiant import FileManager, PathLoader
from ipyradiant.visualization import CytoscapeViewer

Let's start by getting our graph loaded in.

Explain why we need a subgraph (too many nodes causes issues with cytoscape).

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars_sm.ttl"]

In [ ]:
CytoscapeViewer(graph=lw.graph)

## Now we can initiate the `CytoscapeViewer` object, which allows us to get a nice representation of an RDF or networkx graph

### Steps
#### 1. Focus Query
#### 2. Widget class

In [ ]:
from ipyradiant.query.api import build_values
from ipyradiant.query.api import SPARQLQueryFramer
from rdflib import URIRef, Graph

class MetaFocusQuery(type):
    """Metaclass to TODO"""

    _sparql = """
        CONSTRUCT {{
            ?s ?p ?o .
        }} WHERE {{
            ?s ?p ?o .

            VALUES ({}) {{
                {}
            }}
        }}
        ORDER BY DESC(?type)
        LIMIT 10
    """
    values = None

    @property
    def sparql(cls):
        return build_values(cls._sparql, cls.values)
    
    
class FocusQuery(SPARQLQueryFramer, metaclass=MetaFocusQuery):
    values = {"s": [URIRef("https://swapi.co/resource/human/4")]}
    columns = ["s", "p", "o"]

In [ ]:
qres_df = FocusQuery.run_query(lw.graph)
# Dump to rdf graph
rdf_graph = Graph()
for triple in qres_df.values:
    rdf_graph.add(triple)

In [ ]:
import ipywidgets as W
import traitlets as T
import networkx as nx
import ipycytoscape as cyto


class FocusView(W.Box):
    cytoscape_widget = T.Instance(CytoscapeViewer, kw={})
    # focus_node = # TODO what should this be?
    # probably some graph attribute
    
    @T.validate("children")
    def validate_children(self, proposal):
        """
        Validate method for default children.
        This is necessary because @trt.default does not work on children.
        """
        children = proposal.value
        if not children:
            children = (
                self.cytoscape_widget,
            )
        return children

In [ ]:
from IPython.display import JSON, display


class CombinedExample(W.VBox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.json_output = W.Output()
        self.interactive_widget = kwargs["interactive_widget"]
        self.children = [self.interactive_widget, self.json_output]
        self.interactive_widget.cytoscape_widget.on("node", "click", self.load_json)

    def load_json(self, node):
        data = node["data"]
        data.pop("_label", None)
        data.pop("_attrs", None)
        with self.json_output:
            self.json_output.clear_output()
            display(JSON(data))

In [ ]:
fv = FocusView()
fv.cytoscape_widget.include_missing_nodes = True
fv.cytoscape_widget.graph = rdf_graph
fv.cytoscape_widget.layout.width = "100%"

In [ ]:
ce = CombinedExample(interactive_widget=fv.cytoscape_widget)
ce